In [180]:
import pandas as pd
import gensim
import os
import collections
import smart_open
import flask
from flask import Flask
import pickle
import random
from surprise import SVDpp,SVD
from surprise import Dataset
from surprise import Reader
import re
from sklearn.metrics.pairwise import euclidean_distances

app = flask.Flask(__name__)

#pulling in dataset, item list
with open('/Users/deven/Documents/pickleddata/projectfletcher/newdatalist.pkl', 'rb') as picklefile:
    itemdf = pickle.load(picklefile)
with open('/Users/deven/Documents/GitHub/Metis/ProjectFletcher/surprise_data.pkl', 'rb') as picklefile:
    newdf = pickle.load(picklefile)
with open('/Users/deven/Documents/pickleddata/projectfletcher/btrain.pkl', 'rb') as picklefile:
    btrain = pickle.load(picklefile)
with open('doclen.pkl', 'rb') as picklefile:
    doclen = pickle.load(picklefile)

In [181]:
#changing some tea names for easier calls
itemdf = pd.DataFrame(itemdf)
newname=[]
import re

for i in itemdf['Tea Name']:
    line = re.sub('[!@#$\'\",]', '', i)
    newname.append(line)
itemdf['Tea Name'] = newname

#initializing book list
bookt = ['Emma by Jane Austen', 'Persuassion by Jane Austen', 'Sense and Sensibility by Jane Austen',\
        'Poems by William Blake', 'The Little People of the Snow by William Bryant', 'The Adventures of Buster Bear by Thornton Burgress'\
        'Alice in Wonderland by Lewis Carroll','The Ball and the Cross by G.K. Chesterton','The Wisdom of Father Brown by G.K. Chesterton'\
        'The Ball and the Cross by G.K. Chesterton', 'The Parents Assistant by Maria Edgeworth','Moby Dick by Herman Melville',\
        'Paradise Lost by John Milton', 'Shakespeares Works','Shakespeares Works','Shakespeares Works', 'Leaves of Grass by Walt Whitman']

imagelocs = {'Emma by Jane Austen':'static/emma.jpg','Persuassion by Jane Austen':'static/persuassion.jpg','Sense and Sensibility by Jane Austen':'static/sense.jpg',\
'Poems by William Blake':'static/blake.jpg','The Little People of the Snow by William Bryant':'static/bryant.jpg','The Adventures of Buster Bear by Thornton Burgress':'static/the-adventures-of-buster-bear.jpg',\
'The Adventures of Buster Bear by Thornton BurgressAlice in Wonderland by Lewis Carroll':'static/alice.jpg', 'The Ball and the Cross by G.K. Chesterton':'static/ball.jpg','The Wisdom of Father Brown by G.K. Chesterton':'static/fatherbrown.jpg',\
'The Parents Assistant by Maria Edgeworth':'static/edge.jpg','Moby Dick by Herman Melville':'static/moby.jpg','Paradise Lost by John Milton':'static/paradise.jpg',\
'Shakespeares Works':'static/shake.jpg','Shakespeares Works':'static/shake.jpg','Shakespeares Works':'static/shake.jpg','Leaves of Grass by Walt Whitman':'static/leaves.jpg'}



In [81]:
alist= []
for i in itemdf['Flavor Profile Reviews']:
        alist.append(i)
teadf = pd.DataFrame(alist)

In [170]:
#Call function for top ratings
from collections import defaultdict
def get_top_n(teaid,score,qq, n=10):
    # A reader is still needed but only the rating_scale param is requiered.
    qq = pd.concat([qq,pd.DataFrame([[score,teaid, 'user1']], columns = ['Score', 'Tea Name', 'User Name'])], ignore_index=True)
    reader = Reader(rating_scale=(0, 100))
    algo=SVD()
    # The columns must correspond to user id, item id and ratings (in that order).
    data = Dataset.load_from_df(qq[['User Name', 'Tea Name', 'Score']], reader)
    trainset = data.build_full_trainset()
    algo.fit(trainset)

    testset = trainset.build_anti_testset()
    predictions = algo.test(testset)


    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''
    want= []
    for i in predictions:
        if i[0]== 'user1':
            want.append(i)
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in want:
        top_n[uid].append((iid, est))
    print(want)
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    teadist = []
    mindist = []
    eudist=0
    for i in top_n['user1']:
        eudis=(euclidean_distances(teadf.iloc[itemdf[itemdf['Tea Name']==teaid].index,:], \
            teadf.iloc[(itemdf[itemdf['Tea Name']==i[0]].index),:]))
        teadist.append((i[0],eudist))
    mindist = sorted(teadist, key=lambda x:x[1])
    print(teadist)
    print(mindist)
    return mindist[:3]



In [154]:
itemdf[itemdf['Tea Name']==i[0]].index[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [184]:
itemdf.head()

,Flavor Profile Cust,Flavor Profile Reviews,Mouthfeel,Review Adj,Reviews Supertext,Tea Name
0,0,"{'Spicy': 0.0, 'Char': 0.0, 'Sweet': 0.0, 'Nut...",[],"[rich, pu, erh, teaopias, little, selection, p...",A very rich tasting pu erh Teaopias little s...,Tuo Cha Pu-erh
1,0,"{'Spicy': 0, 'Char': 0, 'Sweet': 0, 'Nutty': 0...",[],"[smooth, nice, nutty, taste, aroma, great, din...",Smooth with a nice nutty taste and aroma Gre...,Genmaicha Brown Rice Tea
2,0,"{'Spicy': 0, 'Char': 0, 'Sweet': 0, 'Nutty': 0...",[],[grand],Grand,Lapsang Souchong
3,0,"{'Spicy': 0, 'Char': 0, 'Sweet': 0, 'Nutty': 0...",[],"[ive, first, zingermans, product, fair, right,...",Sadly I think Ive just encountered the first ...,Moonlight White
4,0,"{'Spicy': 0, 'Char': 0, 'Sweet': 0, 'Nutty': 0...",[],"[cest, effectivement, un, bon, puerh, velouté,...",Cest effectivement un bon Puerh velouté gras ...,Pu Er Vrac 28 (1998)


In [171]:
def getBookrec(iid):
    bookrec = gensim.models.doc2vec.Doc2Vec.load('/Users/deven/Documents/pickleddata/projectfletcher/bookrec.bin')
    test_corpus = itemdf[itemdf['Tea Name']==iid]['Review Adj'].values[0]
    inferred_vector = bookrec.infer_vector(test_corpus)
    sims = bookrec.docvecs.most_similar([inferred_vector])
    rec=''
    tot=0
    for ind, i in enumerate(doclen):
        tot+=i
        if sims[0][0]<tot:
            rec = bookt[ind-1]
            break
    print(rec)
    return rec

In [178]:
est_corpus = itemdf[itemdf['Tea Name']=='Chai Matcha']['Review Adj'].values[0]

In [179]:
est_corpus

array([list(['part', 'effort', 'cupboard', 'things', 'im', 'final', 'cup', '“', 'pregnant', 'friend', 'hers', 'haha', 'way', 'interest', 'tea', 'place', 'i', 'less', 'matcha', 'cupboard', 'nanowrimo', 'kind', 'forget', 'things', 'eat', 'sleep', 'ive', 'small', 'bowl', 'cereal', 'considering', 'low', 'blood', 'sugar', 'regular', 'intervals', 'good', 'combo', 'kitchen', 'man', 'hot', 'cocoa', 'okay', 'dinner', 'sugar', 'college', 'dorm', 'standbys', 'cup', 'milk', 'spoonful', 'nutella', 'best', 'hot', 'cocoa', 'concoction', 'thinking', 'good', 'cinnamon', 'little', 'nutmeg', 'chai', 'spices', 'chai', 'matcha', 'chai', 'matcha', 'nutella', 'hot', 'chocolate', 'right', 'greatest', 'thing', 'tons', 'flavors', 'nutella', 'caramel', 'nutella', 'nutella', 'matcha', 'nutella', 'right', 'now', 'well', 'fantastic', 'latte', 'morning', 'pumpkin', 'pie', 'matcha', 'd', 'smell', 'envelope', 'intoxicating\uf04a', 'cinnamon', 'ginger', 'cloves', 'other', 'chai', 'spices', 'amazing', 'ratio', 'matcha',

In [173]:
data = {'example':['Chai Matcha','green','83']}

In [174]:
x = data["example"]
rec1 = get_top_n(x[0],x[2], newdf)
bookrec= getBookrec(x[0])
imgrec = imagelocs[bookrec]
# Put the result in a nice dict so we can send it as json
results = {"tearec1":rec1,"tearec2":rec2,"tearec3":rec3,"bookrec":bookrec, 'img':imgrec}
#return flask.jsonify(results)

#--------- RUN WEB APP SERVER ------------#

# Start the app server on port 80
# (The default website port)
#app.run(host='0.0.0.0')
# app.run(debug=True)

[Prediction(uid='user1', iid='Berryblossom White', r_ui=67.56847518536112, est=60.87953377840121, details={'was_impossible': False}), Prediction(uid='user1', iid='Blood Orange', r_ui=67.56847518536112, est=70.83094768119702, details={'was_impossible': False}), Prediction(uid='user1', iid='Cinnamon Apple', r_ui=67.56847518536112, est=63.67823491211355, details={'was_impossible': False}), Prediction(uid='user1', iid='Orange Passionfruit &amp; Jasmine Green', r_ui=67.56847518536112, est=68.62387506849849, details={'was_impossible': False}), Prediction(uid='user1', iid='Apricot Green', r_ui=67.56847518536112, est=69.43507780589154, details={'was_impossible': False}), Prediction(uid='user1', iid='Vanilla', r_ui=67.56847518536112, est=61.277858592534464, details={'was_impossible': False}), Prediction(uid='user1', iid='Kyoto Sunset', r_ui=67.56847518536112, est=69.53489037836707, details={'was_impossible': False}), Prediction(uid='user1', iid='Green Tea', r_ui=67.56847518536112, est=61.950756

[('Read My Lips', 0), ('Ti Kuan Yin', 0), ('Coconut Pouchong', 0), ('Iron Goddess of Mercy (Ti Kuan Yin)', 0), ('Zingiber Ginger Coconut', 0), ('Coconut Cream Pie', 0), ('Long Life Oolong', 0), ('Japan Matcha Hikari (powdered)', 0), ('Maidens Ecstasy', 0), ('Buttered Rum (organic)', 0)]
[('Read My Lips', 0), ('Ti Kuan Yin', 0), ('Coconut Pouchong', 0), ('Iron Goddess of Mercy (Ti Kuan Yin)', 0), ('Zingiber Ginger Coconut', 0), ('Coconut Cream Pie', 0), ('Long Life Oolong', 0), ('Japan Matcha Hikari (powdered)', 0), ('Maidens Ecstasy', 0), ('Buttered Rum (organic)', 0)]
The Adventures of Buster Bear by Thornton BurgressAlice in Wonderland by Lewis Carroll


In [175]:
print(rec1)
print(results)

[('Read My Lips', 0), ('Ti Kuan Yin', 0), ('Coconut Pouchong', 0)]
{'tearec1': [('Read My Lips', 0), ('Ti Kuan Yin', 0), ('Coconut Pouchong', 0)], 'tearec2': ('Love Tea 7', 0), 'tearec3': ('Dawn', 0), 'bookrec': 'The Adventures of Buster Bear by Thornton BurgressAlice in Wonderland by Lewis Carroll', 'img': 'static/alice.jpg'}
